In [29]:
import pandas as pd
import numpy as np
from collections import defaultdict
from prepare_data import preparing, get_word_list, get_embedding, extract_features, final_prepare
from keras.preprocessing.text import Tokenizer
from NLP import extract_nlp_features
import Non_NLP

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout,BatchNormalization
from tensorflow.keras.layers import concatenate, add, Lambda,multiply, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score,precision_score

In [4]:
import importlib
importlib.reload(Non_NLP)

<module 'Non_NLP' from '/Users/sizhenhan/Documents/quora-question-pairs/Non_NLP.py'>

In [5]:
data = pd.read_csv("train.csv")

In [6]:
X_train, X_test, y_train, y_test = preparing(data)

In [7]:
words = get_word_list(X_train)
embeddings = get_embedding(words)
words = embeddings.keys()

In [8]:
train1,train2, features_train = extract_features(X_train,words)
test1,test2, features_test = extract_features(X_test,words)

In [9]:
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(np.append(train1, train2))

In [10]:
data_train1, data_train2 = final_prepare(train1,train2,tokenizer)
data_test1, data_test2 = final_prepare(test1,test2,tokenizer)

In [11]:
y_train = np.array(y_train)
embedding_matrix = np.zeros((len(words)+1, 300))

for i, word in enumerate(words):
    embedding_matrix[i] = embeddings.get(word)

In [12]:
features_train

array([[0., 1., 0., 0.],
       [0., 2., 0., 0.],
       [0., 2., 0., 0.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 5., 0., 0.]])

## NLP Features 

In [13]:
X_train2 = X_train.copy()
X_test2 = X_test.copy()

In [14]:
X_nlp_features_train = extract_nlp_features(X_train2)
X_nlp_features_test = extract_nlp_features(X_test2)

In [15]:
X_nlp_features_train.head()

,common_word_min,common_word_max,common_stop_min,common_stop_max,common_token_min,common_token_max,last_word_equal,first_word_equal,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
70052,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,8.0,11.0,43,43,31,52,0.184211
321015,0.499988,0.285710,0.599988,0.374995,0.555549,0.333331,0.0,1.0,6.0,12.0,60,45,46,49,0.180000
268021,0.428565,0.374995,0.000000,0.000000,0.230767,0.199999,0.0,0.0,2.0,14.0,60,51,28,29,0.219178
312808,0.999967,0.999967,0.666644,0.499988,0.833319,0.714276,1.0,1.0,1.0,6.5,95,85,88,81,0.515152
265818,0.999900,0.999900,0.999950,0.666644,0.999967,0.749981,1.0,1.0,1.0,3.5,100,92,92,83,0.615385


## Non NLP features

In [16]:
X_train3 = X_train.copy()
X_test3 = X_test.copy()

In [17]:
X_nonnlp_features_train, X_nonnlp_features_test = Non_NLP.extract_nonnlp_features(X_train3,X_test3)

In [18]:
X_nonnlp_features_train.head()

,min_kcore,max_kcore,common_neighbor_ratio,common_neighbor_count,min_freq,max_freq
70052,0,0,0.0,0.0,1,5
321015,0,2,0.0,0.0,1,4
268021,0,0,0.0,0.0,1,1
312808,0,2,0.5,1.0,2,2
265818,0,3,0.0,0.0,1,2


## Merge Features

In [19]:
features_train = np.hstack((features_train, X_nlp_features_train, X_nonnlp_features_train))
features_test = np.hstack((features_test, X_nlp_features_test, X_nonnlp_features_test))

## Training 

In [32]:
input1 = Input(shape=(data_train1.shape[1],))
embedding1 = Embedding(len(words)+1,300,weights=[embedding_matrix],
                       input_length=data_train1.shape[1],trainable=False)(input1)
x1 = LSTM(75, recurrent_dropout=0.2)(embedding1)

input2 = Input(shape=(data_train1.shape[1],))
embedding2 = Embedding(len(words)+1,300,weights=[embedding_matrix],
                       input_length=data_train1.shape[1],trainable=False)(input2)
x2 = LSTM(75, recurrent_dropout=0.2)(embedding2)

input3 = Input(shape=(features_train.shape[1],))
dense_feature = BatchNormalization()(input3)
dense_feature = Dense(200, activation="relu")(dense_feature)
dense_feature = Dropout(0.2)(dense_feature)

addition = add([x1, x2])
x2_negative = Lambda(lambda x: -x)(x2)
subtraction = add([x1, x2_negative])
subtraction = multiply([subtraction, subtraction])
merged = concatenate([subtraction, addition])
merged = Dropout(0.4)(merged)

merged = concatenate([merged, dense_feature])
merged = BatchNormalization()(merged)
merged = GaussianNoise(0.1)(merged)

merged = Dense(150, activation="relu")(merged)
merged = Dropout(0.2)(merged)
merged = BatchNormalization()(merged)

out = Dense(1, activation="sigmoid")(merged)

model = Model(inputs=[input1, input2, input3], outputs=out)
model.compile(loss="binary_crossentropy",
                  optimizer="nadam",metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 30, 300)      1029600     input_12[0][0]                   
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 30, 300)      1029600     input_11[0][0]                   
____________________________________________________________________________________________

In [33]:
model.fit([data_train1, data_train2, features_train], y_train,
                     epochs=15, batch_size= 512, verbose=1)

Train on 323432 samples
Epoch 1/15
323432/323432 [==============================] - 141s 436us/sample - loss: 0.3077 - accuracy: 0.8604
Epoch 2/15
323432/323432 [==============================] - 133s 412us/sample - loss: 0.2809 - accuracy: 0.8743
Epoch 3/15
323432/323432 [==============================] - 133s 411us/sample - loss: 0.2738 - accuracy: 0.8774
Epoch 4/15
323432/323432 [==============================] - 135s 418us/sample - loss: 0.2674 - accuracy: 0.8801
Epoch 5/15
323432/323432 [==============================] - 135s 417us/sample - loss: 0.2623 - accuracy: 0.8832
Epoch 6/15
323432/323432 [==============================] - 137s 423us/sample - loss: 0.2594 - accuracy: 0.8841
Epoch 7/15
323432/323432 [==============================] - 137s 423us/sample - loss: 0.2555 - accuracy: 0.8855
Epoch 8/15
323432/323432 [==============================] - 140s 432us/sample - loss: 0.2527 - accuracy: 0.8872
Epoch 9/15
323432/323432 [==============================] - 136s 422us/sample - 

In [34]:
pred = model.predict([data_test1, data_test2, features_test])

In [35]:
preds = []
for p in pred:
    if p > 0.5:
        preds.append(1)
    else:
        preds.append(0)

In [36]:
accuracy_score(preds,y_test)

0.8881990650275792

In [37]:
precision_score(preds,y_test,pos_label=1)

0.8113207547169812

In [38]:
input1 = Input(shape=(data_train1.shape[1],))
embedding1 = Embedding(len(words)+1,300,weights=[embedding_matrix],
                       input_length=data_train1.shape[1],trainable=False)(input1)
x1 = LSTM(75, recurrent_dropout=0.2)(embedding1)

input2 = Input(shape=(data_train1.shape[1],))
embedding2 = Embedding(len(words)+1,300,weights=[embedding_matrix],
                       input_length=data_train1.shape[1],trainable=False)(input2)
x2 = LSTM(75, recurrent_dropout=0.2)(embedding2)

addition = add([x1, x2])
x2_negative = Lambda(lambda x: -x)(x2)
subtraction = add([x1, x2_negative])
subtraction = multiply([subtraction, subtraction])
merged = concatenate([subtraction, addition])
merged = Dropout(0.4)(merged)

merged = BatchNormalization()(merged)
merged = GaussianNoise(0.1)(merged)

merged = Dense(150, activation="relu")(merged)
merged = Dropout(0.2)(merged)
merged = BatchNormalization()(merged)

out = Dense(1, activation="sigmoid")(merged)

model2 = Model(inputs=[input1, input2], outputs=out)
model2.compile(loss="binary_crossentropy",
                  optimizer="nadam",metrics=['accuracy'])
model2.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 30, 300)      1029600     input_15[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 30, 300)      1029600     input_14[0][0]                   
____________________________________________________________________________________________

In [39]:
model2.fit([data_train1, data_train2, features_train], y_train,
                     epochs=15, batch_size= 512, verbose=1)

Train on 323432 samples
Epoch 1/15
323432/323432 [==============================] - 134s 414us/sample - loss: 0.6181 - accuracy: 0.6603
Epoch 2/15
323432/323432 [==============================] - 132s 407us/sample - loss: 0.5471 - accuracy: 0.7201
Epoch 3/15
323432/323432 [==============================] - 133s 410us/sample - loss: 0.5199 - accuracy: 0.7376
Epoch 4/15
323432/323432 [==============================] - 133s 410us/sample - loss: 0.4990 - accuracy: 0.7511
Epoch 5/15
323432/323432 [==============================] - 132s 409us/sample - loss: 0.4832 - accuracy: 0.7607
Epoch 6/15
323432/323432 [==============================] - 135s 418us/sample - loss: 0.4696 - accuracy: 0.7695
Epoch 7/15
323432/323432 [==============================] - 131s 404us/sample - loss: 0.4580 - accuracy: 0.7767
Epoch 8/15
323432/323432 [==============================] - 133s 412us/sample - loss: 0.4467 - accuracy: 0.7834
Epoch 9/15
323432/323432 [==============================] - 133s 410us/sample - 

In [40]:
pred2 = model2.predict([data_test1, data_test2])
preds = []
for p in pred2:
    if p > 0.5:
        preds.append(1)
    else:
        preds.append(0)
accuracy_score(preds,y_test)

0.8011204828217369

In [41]:
precision_score(preds,y_test,pos_label=1)

0.6694286379820971